# GRPC Inference

### Setup

In [ ]:
!pip list | grep -e grpcio -e protobuf

In [ ]:
!pip install grpcio==1.56.0 grpcio-tools==1.33.2 protobuf==3.20.3

In [ ]:
!pip list | grep -e grpcio -e protobuf

In [ ]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033

textencoder_model_name = 'textencoder'
unet_model_name = 'unet'
vaeencoder_model_name = 'vaeencoder'
vaedecoder_model_name = 'vaedecoder'

### Inspecting the gRPC Endpoint

Let's check out the gRPC endpoint's model metadata.

In [ ]:
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

# request = grpc_predict_v2_pb2.ModelMetadataRequest(name=vaeencoder_model_name)
# response = stub.ModelMetadata(request)
# print(response)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=vaedecoder_model_name)
response = stub.ModelMetadata(request)
print(response)


### Request Function

Builds and submits our gRPC request.

In [ ]:
import torch

def vae_decoder_grpc_request(latent_sample):
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "latent_sample"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 4, 64, 64])
    arr = latent_sample.flatten()
    inputs[0].contents.fp32_contents.extend(arr)

    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = vaedecoder_model_name
    request.inputs.extend(inputs)

    response = stub.ModelInfer(request)
    out_sample = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)

    return torch.tensor(out_sample.reshape([1, 3, 512, 512]))

### Run the Request

In [ ]:
import numpy as np
import torch

from IPython.display import Image

latents = np.load("latents.npy")

In [ ]:
import torch
import numpy as np


image_tensor = vae_decoder_grpc_request(latents)

In [ ]:
print(image_tensor.dtype)
print(image_tensor.shape)
print(image_tensor)

In [ ]:
image_tensor.sample

In [ ]:
print(image_tensor.dtype)
print(image_tensor.shape)
print(image_tensor)

In [ ]:
from PIL import Image

def image_to_pil(image_tensor):    
    image = (image_tensor / 2 + 0.5).clamp(0, 1).squeeze()
    image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
    images = (image * 255).round().astype("uint8")
    image = Image.fromarray(image)
    return image

In [ ]:
image = (image_tensor / 2 + 0.5).clamp(0, 1).squeeze()
image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
images = (image * 255).round().astype("uint8")
image = Image.fromarray(image)
image

In [ ]:
img = image_to_pil(image_tensor)

In [ ]:
img